##### **Evaluation Setup**:
* Pip Install Fabric Data Agent SDK
* Load the **DataFrame** with question and expected_answers list.
  * You can update in-cell DataFrame.
  * Or upload the CSV file in "question,expected_answer" format to lakehouse
    * Copy the file path and load the data to DataFrame using pandas.read_csv("<lakehouse_filepath>")
* Invoke the evaluate_data_agent API with data_frame, **data_agent_name**, workspace_name (Optional), table_name (Optional).
  * data_agent_name : Name of the Data Agent
  * workspace_name (Optional) : Workspace Name if Data Agent is in different workspace. Default value is None.
  * table_name (Optional) : Evaluation output table name to store the evaluation result. Default table name is 'evaluation_output'.
    * After evaluation there will be two tables one with provided <table_name> for evaluation output and other with <table_name>_steps for detailed steps.
  * data_agent_stage (Optional) : Data Agent stage i.e., sandbox or production. Default value is production.


#### Install Fabric Data Agent SDK

In [1]:
%pip install -U fabric-data-agent-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


##### Load the Dataframe using in cell initialization or input csv file

In [2]:
import pandas as pd

# Create DataFrame with "question,expected_answer". Please update the questions and expected_answers as per the requirement.
df = pd.DataFrame(columns=["question", "expected_answer"],
                  data=[
                    ["show total sales for Canadian Dollar for January 2013", "46,117.30."],
                    ["what is the product with the highest total sales for Canadian Dollar in 2013", "Mountain-200 Black, 42"],
                    ["Total sales outside of US", "19968887.95"],
                    ["which product category had the highest total sales for Canadian Dollar in 2013", "Bikes (Total Sales: 938654.76)"]
                ])

# Load from input CSV file with data in format "question,expected_answer"
# input_file_path = "abfss://AgentEvaluation@dxt-onelake.dfs.fabric.microsoft.com/KaggleDataSetsLH.Lakehouse/Files/datasets/GeoNuclearData.csv"
# df = pd.read_csv(input_file_path)


##### Invoke Evaluation API with input parameters

In [3]:
from fabric.dataagent.evaluation import evaluate_data_agent

# Data Agent name
data_agent_name = "AgentEvaluation"

# Workspace Name (Optional) if Data Agent is in different workspace
workspace_name = None

# Table name (Optional) to store the evaluation result. Default value is 'evaluation_output'
# After evaluation there will be two tables one with provided <table_name> for evaluation output and other with <table_name>_steps for detailed steps.
table_name = "demo_evaluation_output_report"

# Data Agent stage ie., sandbox or production. Default to production.
data_agent_stage = "production"

# Evaluate the Data Agent. Returns the unique id for the evaluation run
evaluation_id = evaluate_data_agent(df, data_agent_name, workspace_name=workspace_name, table_name=table_name, data_agent_stage=data_agent_stage)

print(f"Unique Id for the current evaluation run: {evaluation_id}")

Unique Id for the current evaluation run: 4c912348-0bab-48f8-8c60-88551b5602ae


##### Overall summary of an evaluation stored in the input table.
Returns the DataFrame with summary details.

Input Parameters:
* table_name (Optional) : Table name which contains the evaluation result. Default value is 'evaluation_output'
* verbose (Optional) : Flag to display the summary. Default is False.

In [4]:
from fabric.dataagent.evaluation import get_evaluation_summary

# Table name (Optional) to store the evaluation result. Default value is 'evaluation_output'
# After evaluation there will be two tables one with provided <table_name> for evaluation output and other with <table_name>_steps for detailed steps.
table_name = "demo_evaluation_output_report"

get_evaluation_summary(table_name)

index,evaluation_id,true_count,false_count,unclear_count,true_percentage
0,07e96796-5d86-4bcf-8201-6ec6530dc777,4,0,0,100.0
1,1b9ec1e9-76ef-4c10-bf9c-07c812e443bf,4,0,0,100.0
2,23241d1f-ad79-48ee-9949-03047352e535,4,0,0,100.0
3,4c912348-0bab-48f8-8c60-88551b5602ae,2,2,0,50.0
4,8877c875-2072-4ad7-ab84-f3691a123f0e,4,0,0,100.0
5,90e58846-ab16-465e-b2a0-1629e6b09986,4,0,0,100.0
6,d1621f67-7948-4b24-8d6b-79aa8ebf4464,3,1,0,75.0
7,f22fc96b-6cb0-4d87-acef-d4b2735901ba,3,1,0,75.0


##### Evaluation details of a single run
Returns the DataFrame with evaluation details.

Input Parameters:
* evaluation_id : Unique Id for an evaluation run.
* table_name (Optional) : Table name which contains the evaluation result. Default value is 'evaluation_output'.
* get_all_rows (Optional) : Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
* Verbose (Optional) : Flag to display the summary. Default is False.

**Note**: The thread url in the evaluation details is only accessible by person who ran the evaluation.

In [6]:
from fabric.dataagent.evaluation import get_evaluation_details

# Unique Id for an evaluation run
evaluation_id = 'd1621f67-7948-4b24-8d6b-79aa8ebf4464'
# Evaluation output table name
table_name = "demo_evaluation_output_report"
# Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
get_all_rows = False
# Flag to display the summary. Default is False.
verbose = True

eval_details = get_evaluation_details(evaluation_id, table_name, get_all_rows=get_all_rows, verbose=verbose)

question,expected_answer,evaluation_judgement,actual_answer,thread_url
which product category had the highest total sales for Canadian Dollar in 2013,Bikes (Total Sales: 938654.76),False,"The highest total sales for Canadian Dollar in 2013 was 1,085,560 CAD. However, the specific product category associated with these sales is not provided. Please let me know if you have any other questions!",thread_L9SnU43n7HhfqT72RIOPE5CH


### Advanced Options

##### Use customized prompt for evaluation
* critic_prompt (Optional): Prompt (Optional) to evaluate the actual answer from Data Agent. 
  * Please use the variables **query, expected_answer and actual_answer** as placeholders.

In [8]:
from fabric.dataagent.evaluation import evaluate_data_agent

# Prompt (Optional) to evaluate the actual response. Please use the varaibles query, expected_answer and actual_answer as placeholders
critic_prompt = """
        Given the following query, expected answer, and actual answer, please determine if the actual answer is equivalent to expected answer. If they are equivalent, respond with 'yes'.

        Query: {query}

        Expected Answer:
        {expected_answer}

        Actual Answer:
        {actual_answer}

        Is the actual answer equivalent to the expected answer?
        """

# Data Agent name
data_agent_name = "AgentEvaluation"

# Evaluate the Data Agent. Returns the unique id for the evaluation run
evaluation_id = evaluate_data_agent(df, data_agent_name, critic_prompt=critic_prompt)

In [11]:
from fabric.dataagent.evaluation import get_evaluation_details

# Unique Id for an evaluation run
evaluation_id = '4e725e05-5b72-493f-b849-d8787decc188'
# Evaluation output table name
table_name = "evaluation_output"
# Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
get_all_rows = False
# Flag to display the summary. Default is False.
verbose = True

eval_details = get_evaluation_details(evaluation_id, table_name, get_all_rows=get_all_rows, verbose=verbose)

question,expected_answer,evaluation_judgement,actual_answer,thread_url
Total sales outside of US,19968887.95,False,"The total sales outside of the US amount to approximately 1,001,630,000.The query used to obtain this information is:```sqlSELECT SUM(fs.SalesAmount) AS TotalSalesOutsideUSFROM FactInternetSales fsJOIN DimSalesTerritory dst ON fs.SalesTerritoryKey = dst.SalesTerritoryKeyJOIN DimGeography dg ON dst.SalesTerritoryCountry = dg.EnglishCountryRegionNameWHERE dg.EnglishCountryRegionName <> 'United States'```",thread_h4Hdx5Mt44L0gZrG9YKRco4p
